In [ ]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import squidpy as sq

from scipy.spatial import cKDTree

In [ ]:
import liana as li

In [ ]:
# slide names and x, y offsets
slides = {'A1':[3.5, 18],
          'B1':[3, 10.5],
          'C1':[1, 13.5],}

In [ ]:
## load slide
slide = "B1"
exp = "V11L12-109" # V11T17-102 V11L12-109 V11T16-085
slide_path = f"data/VicariEtAl/sma/{exp}/{exp}_{slide}/output_data"
rna_path = slide_path + f"/{exp}_{slide}_RNA/outs/"

In [ ]:
rna = sq.read.visium(rna_path, counts_file="filtered_feature_bc_matrix.h5")
rna.obs[['x', 'y']] = rna.obsm['spatial']

msi_raw = pd.read_csv(slide_path + f"/{exp}_{slide}_MSI/{exp}_{slide}.Visium.FMP.220826_smamsi.csv")


In [ ]:
# set x and y columns as index, concat with _ as separator
msi_raw.index = msi_raw[['x', 'y']].astype(str).agg('_'.join, axis=1)

In [ ]:
# Only for mouse
lesion = pd.read_csv(rna_path + "/lesion.csv").set_index("Barcode")
region = pd.read_csv(rna_path +  "/region.csv").set_index("Barcode")
rna.obs = rna.obs.join(lesion)
rna.obs = rna.obs.join(region)

MSI

In [ ]:
msi = msi_raw.copy().drop(['x', 'y'], axis=1)
msi = sc.AnnData(X=msi.values,
                 obs=pd.DataFrame(index=msi.index),
                 var=pd.DataFrame(index=msi.columns))

In [ ]:
# copy raw intensities
msi.layers['raw'] = msi.X

# normalize and cluster
sc.pp.normalize_total(msi, target_sum=1e4)
sc.pp.log1p(msi)
sc.pp.scale(msi)
sc.pp.pca(msi)
sc.pp.neighbors(msi, n_neighbors=10, n_pcs=20)
sc.tl.leiden(msi, resolution=0.00001)

In [ ]:
# get positions; NOTE: x and y are mirrored in the msi data
positions = msi_raw[['y', 'x']].values.copy()

# estimate scaling factor
sfactor = rna.uns['spatial'][f"{exp}_{slide}"]['scalefactors']['fiducial_diameter_fullres']
sfactor

In [ ]:
# inverse y
positions[:, 0] = (positions[:, 0].max() - positions[:, 0])
# offset
x_offset = slides[slide][0]
y_offset = slides[slide][1]

positions[:, 0] = positions[:, 0] + x_offset
positions[:, 1] = positions[:, 1] + y_offset

msi.obs[['array_row', 'array_col']] = positions

# scale
positions = positions * sfactor * 0.95

In [ ]:
msi.uns = rna.uns.copy()

In [ ]:
msi.obsm['spatial'] = positions
msi.obs[['x', 'y']] = positions

In [ ]:
# remove cells that are not in the tissue
msi = msi[msi.obs['leiden'] != '1', :]

In [ ]:
sc.pl.spatial(msi, color="leiden")

In [ ]:
import muon as mu

In [ ]:
# msi.obs.index = msi.obsm['spatial']

In [ ]:
mdata = mu.MuData({'rna':rna, 'msi':msi})

In [ ]:
sq.pl.spatial_scatter(msi, color=['674.2805']) # plot dopamine

In [ ]:
# swap back to raw intensities
mdata.mod['msi'].X = mdata.mod['msi'].layers['raw'].copy()
del mdata.mod['msi'].layers['raw']

In [ ]:
mdata.write_h5mu(slide_path + "/sma.h5mu")

In [ ]:
mdata.obs